In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize 
import matplotlib.pyplot as plt 
import warnings

In [2]:
def kalman_filter_two_factor(param, init_state_vect, init_state_cov, futuredata, matur, dt):
    """
    Kalman Filter para um processo de preço com dois fatores.
    
    Retorna a pontuação LogLScore para fins de minimização.
    """

    # Inicialização dos parâmetros
    kappa, sigmachi, riskchi, muxi, sigmaxi, riskxi, rho = param[:7]
    s = param[7:]  # erros, conforme Schwartz e Smith

    # Definição das variáveis na equação de transição
    Ci = np.array([0, muxi * dt]).reshape(-1, 1)
    Gi = np.array([[np.exp(-kappa * dt), 0], [0, 1]])

    xx = (1 - np.exp(-2 * kappa * dt)) * (sigmachi ** 2 / (2 * kappa))

    xy = (1 - np.exp(-kappa * dt)) * rho * sigmachi * sigmaxi / kappa
    yy = sigmaxi ** 2 * dt
    Q = np.array([[xx, xy], [xy, yy]])


    # Definição das variáveis na equação de medição
    first_at = (muxi - riskxi) * matur
    second_at = -(1 - np.exp(-kappa * matur)) * (riskchi / kappa)
    third_at = 0.5 * ((1 - np.exp(-2 * kappa * matur)) * (sigmachi ** 2 / kappa) +
                      sigmaxi ** 2 * matur +
                      2 * (1 - np.exp(-kappa * matur)) * rho * sigmaxi * sigmachi / kappa)
    
    di = (first_at + second_at + third_at).reshape(-1, 1)
    Fi = np.vstack([np.exp(-kappa * matur), np.ones(len(matur))]).T
    R = np.eye(Q.shape[0])
    V = np.diag(s)

    # Variáveis para armazenar os resultados do filtro de Kalman
    vtgivtvec = np.zeros_like(futuredata)
    Xtgivtvec = np.zeros((futuredata.shape[0], 2))

    # Inicialização
    Xtgivt = init_state_vect.reshape(-1, 1)
    Ptgivt = init_state_cov

    result_second_term_logl = np.zeros(futuredata.shape[0])
    result_first_term_logl = np.zeros(futuredata.shape[0])

    for i in range(futuredata.shape[0]):
        # Passos de previsão
        Xtgivtmin1 = Ci + Gi @ Xtgivt
        Ptgivtmin1 = Gi @ Ptgivt @ Gi.T + R @ Q @ R.T

        # Passos de medição (atualização)
        yt = futuredata[i, :].reshape(-1, 1)
        vtgivtmin1 = yt - (di + Fi @ Xtgivtmin1)

        Htgivtmin1 = Fi @ Ptgivtmin1 @ Fi.T + V
    
        #Kt = Ptgivtmin1 @ Fi.T @ np.linalg.inv(Htgivtmin1)
        Kt = Ptgivtmin1 @ Fi.T @ np.linalg.pinv(Htgivtmin1)

        Xtgivt = Xtgivtmin1 + Kt @ vtgivtmin1
        Ptgivt = Ptgivtmin1 - Kt @ Fi @ Ptgivtmin1


        result_second_term_logl[i] = (vtgivtmin1.T @ np.linalg.pinv(Htgivtmin1) @ vtgivtmin1).item()
        #result_second_term_logl[i] = vtgivtmin1.T @ np.linalg.inv(Htgivtmin1) @ vtgivtmin1

        result_first_term_logl[i] = np.linalg.det(Htgivtmin1)

        vtgivt = yt - (di + Fi @ Xtgivt)
        vtgivtvec[i, :] = vtgivt.flatten()
        Xtgivtvec[i, :] = Xtgivt.flatten()
    
    logl_score = -0.5 * np.sum(np.log(2 * np.pi * result_first_term_logl)) - 0.5 * np.sum(result_second_term_logl)

    return -logl_score, Xtgivtvec


In [3]:
# Carregar os dados
data = pd.read_csv('price_log_1_3_5_7_9_13.csv', header=None).values

# Definir parâmetros iniciais
deltat = 1 / 52
matur = np.array([1/12, 3/12, 5/12, 7/12, 9/12, 13/12])

kappa = 1
sigmachi = 0.01
riskchi = 0.01
muxi = 0.01
sigmaxi = 0.01
riskxi = 0.01
rho = 0.1
s = 0.001

# Criando o vetor de parâmetros iniciais
param_initial = np.concatenate(([kappa, sigmachi, riskchi, muxi, sigmaxi, riskxi, rho], np.full(data.shape[1], s)))

# Inicialização do vetor e matriz de covariância do estado
init_state_vect = np.array([0.28, 3.5])  # [chi, xi]
init_state_cov = np.eye(2) * 0.01

#################################################################################
# Setting up optimization bounds
#lowerbound = np.concatenate(([0.001, 0, -np.inf, -np.inf, 0, -np.inf, -1], np.full(data.shape[1], 1e-8)))
#upperbound = np.concatenate(([np.inf, np.inf, np.inf, np.inf, np.inf, np.inf, 1], np.full(data.shape[1], np.inf)))

lowerbound = np.array([0.001, 0.001, -100, -100, 0.001, -100, -1.0] + [1e-6] * data.shape[1])
upperbound = np.array([100, 100, 100, 100, 100, 100, 1.0] + [100] * data.shape[1])
#################################################################################
bounds = list(zip(lowerbound, upperbound))

# Função objetivo para o otimizador (Máxima Verossimilhança com Kalman Filter)
logl_kalman = lambda param: kalman_filter_two_factor(param, init_state_vect, init_state_cov, data, matur, deltat)[0]

with warnings.catch_warnings():
    warnings.simplefilter("ignore", RuntimeWarning)
# Executar otimização (MLE - Maximum Likelihood Estimation)
#result = minimize(logl_kalman, param_initial, method='trust-constr', bounds=bounds, options={'disp': True, 'maxiter': 1000})
    result = minimize(logl_kalman, param_initial, method='SLSQP', bounds=bounds, options={'disp': True, 'maxiter': 1000})  # melhores resultados
#result = minimize(logl_kalman, param_initial, method='L-BFGS-B', bounds=bounds, options={'disp': True, 'maxiter': 1000})
#result = minimize(logl_kalman, param_initial, method='trust-exact', bounds=bounds, options={'disp': True, 'maxiter': 1000}) 

# Parâmetros otimizados
param_optimized = result.x
logL = result.fun

logl_score, Xtgivtvec = kalman_filter_two_factor(param_optimized, init_state_vect, init_state_cov, data, matur, deltat)

# Exibir os resultados
param_df = {
    'Parameters': ['kappa', 'sigma_chi', 'risk_chi', 'mu_xi', 'sigma_xi', 'risk_xi', 'rho', 'chi0', 'xi0'],
    'initial': [kappa, sigmachi, riskchi, muxi, sigmaxi, riskxi, rho, init_state_vect[0], init_state_vect[1]],
    'optimized': [param_optimized[0], param_optimized[1], param_optimized[2], param_optimized[3], 
                  param_optimized[4], param_optimized[5],param_optimized[6], Xtgivtvec[-1, 0], Xtgivtvec[-1, 1]]
}
pd.DataFrame(param_df)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -14503.574174939424
            Iterations: 27
            Function evaluations: 387
            Gradient evaluations: 23


,Parameters,initial,optimized
0,kappa,1.00,3.771950
1,sigma_chi,0.01,0.287692
2,risk_chi,0.01,-0.202457
3,mu_xi,0.01,-0.161038
4,sigma_xi,0.01,0.222824
5,risk_xi,0.01,-0.092241
6,rho,0.10,0.398284
7,chi0,0.28,0.049572
8,xi0,3.50,4.207751
